<a href="https://colab.research.google.com/github/Lapitel/llama2-train/blob/main/convertXlsToJsonl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import os
import openpyxl
from bs4 import BeautifulSoup
import json

def getFileNames(directory):
  fileNames = []
  for root, dirs, files in os.walk("."):
    for file in files:
      if file.endswith('.xlsx'):
        path = os.path.join(root, file)
        fileNames.append(path)
  return fileNames

def parseExcelToJsonl(jsonl, file):
  # xlsx 파일 열기
  wb = openpyxl.load_workbook(file)  # 'your_file.xlsx'에는 읽고자 하는 엑셀 파일의 경로가 들어가야 합니다.
  sheet = wb.active

  line  = {"id":"","instruction":"", "input":"", "output":""}
  question = ''
  answer = ''
  irqa = ''
  cnt = 0

  for row in sheet.iter_rows(values_only=True):
      if row[0] == '질문': continue
      if row[0] != None:
        question = row[0]
        #html 태그 제거
        soup = BeautifulSoup(row[1].replace('<br />','\n'), 'html.parser')
        answer = soup.get_text()

        cnt+=1
        line['id'] = "irqa_" + str(cnt)
        line['instruction'] = question
        line['output'] = answer
        jsonl += str(json.dumps(line, ensure_ascii=False)) + '\n'

      if row[2] != None:
        cnt+=1
        irqa=row[2]
        line['id'] = "irqa_" + str(cnt)
        line['instruction'] = irqa
        line['output'] = answer
        # json_line = json.dumps(line, ensure_ascii=False)
        # print(json_line)
        jsonl += str(json.dumps(line, ensure_ascii=False)) + '\n'

  return jsonl


fileNames = getFileNames('.')
jsonl = ''
for file in fileNames :
  jsonl = parseExcelToJsonl(jsonl, file)
# jsonl = parseExcelToJsonl(jsonl, fileNames[0])


# jsonl 파일로 변환
with open('irqa_datasets.jsonl', 'w') as f:
    f.write(jsonl)